
# Des posters enrichissants


## Description du problème


Un étudiant arpente les couloirs du [LIFO](http://www.univ-orleans.fr/lifo/) (le Laboratoire d'Informatique Fondamentale d'Orléans) où sont exposés de magnifiques posters, réalisés par des chercheurs du laboratoire. En voici trois exemples :

<table style="width:100%">
  <tr>
    <th><img alt="exemple poster 1" src="http://www.univ-orleans.fr/lifo/Members/Mathieu.Liedloff/temp/analgo/posters/mc.jpg" width="235"></th>
    <th><img alt="exemple poster 1" src="http://www.univ-orleans.fr/lifo/Members/Mathieu.Liedloff/temp/analgo/posters/ap.jpg" width="235"></th>
    <th><img alt="exemple poster 1" src="http://www.univ-orleans.fr/lifo/Members/Mathieu.Liedloff/temp/analgo/posters/ms.jpg" width="235"></th>
  </tr>
</table> 

<BR>
<div class="alert alert-info">
Pour abstraire le problème, nous notons $n$ le nombre total de posters présentés dans les couloirs. Ces posters sont numérotés $p_1, p_2, \dots , p_n$. Chaque poster $p_i$ ($i \in \{1,2, \dots, n \}$) demande un *temps de lecture* $t_i$ et apporte un *enrichissement intellectuel* $e_i$.
</div>

Un étudiant arrive $m$ minutes en avance pour un rendez-vous avec un enseignant-chercheur. Il se demande quels posters il devrait lire. Son objectif est de maximiser son enrichissement intellectuel tout en ne dépassant pas les $m$ minutes de lecture. S'il dépasse les $m$ minutes de lecture, il rate son rendez-vous. Il faut donc l'aider à bien choisir les posters.

Voici un exemple d'instance du problème :

In [59]:
n = 25 # nombre de posters affichés dans les couloirs
# temps  de lecture et enrichissement de chaque poster (t[k],e[k] sont les info pour le poster k)
t = [16, 15, 26, 36, 12, 13, 35, 25, 22, 7, 13, 15, 25, 14, 26, 17, 25, 13, 12, 11, 22, 33, 15, 13, 15]
e = [15, 9, 7, 18, 14, 15, 16, 18, 8, 14, 8, 11, 8, 3, 9, 15, 19, 1, 2, 3, 3, 13, 13, 14, 2]
m = 70 # nombre de minutes avant le rendez-vous

## Résolution du problème: une approche récursive


Comme première approche, on voudrait résoudre le problème *récursivement*.
Notons `ResolPoster(t,e,i,tps)` la fonction qui attend en paramètre:
- une liste `t` des temps de lecture des posters;
- une liste `e` représentant l'enrichissement intellectuel des posters;
- un entier `i` compris entre $0$ et le nombre $n$ de posters;
- un entier `tps` compris entre $0$ et le temps $m$ disponible pour la lecture.

La fonction `ResolPoster` doit retourner la valeur maximum de l'enrichissement intellectuel  si on se restreint aux `i` premiers posters et qu’on s’autorise un temps maximimum de lecture `tps`.

Pour écrire cette fonction, intéressez-vous aux questions suivantes :
1. Que devrait retourner `ResolPoster(t,e,i,tps)` lorsque `i = 0` ?
2. Que devrait retourner `ResolPoster(t,e,i,tps)` lorsque `tps = 0` ?
3. Si on suppose `i > 0` et `tps > 0`, établir la valeur qui devrait être retournée. Pour calculer cette valeur, des appels récursifs à `ResolPoster` sont certainement nécessaires, mais avec quels paramètres pour `i` et pour `tps` lors de ces appels ?


<div class="alert alert-danger" role="alert">
Une fois ces questions en tête, écrivez en Python la fonction `ResolPoster`. Un appel à `ResolPoster(t,e,n,m)` devrait retourner une solution optimale au problème.
Pour l'instance précédente, nous obtenons 76. Et vous, qu'obtenez-vous ?
</div>

In [60]:
def ResolPoster(t,e,i,tps):
    d = i-1
    if i == 0 or tps == 0:
        return 0
    else:
        if t[d] > tps :
            return (ResolPoster(t,e,i-1,tps))
        else:
            return max (ResolPoster(t,e,i-1,tps-t[d])+e[d],ResolPoster(t,e,i-1,tps))
print(ResolPoster(t,e,n,m))

76


La version récursive calcule une solution correcte au problème, mais elle souffre de lenteur car un même sous-problème peut être plusieurs fois résolus. Cela se traduit par de multiples appels à `ResolPoster` avec des paramètres identiques.

<div class="alert alert-danger" role="alert">
Modifiez votre fonction `ResolPoster` pour qu'elle compte le nombre de fois qu'elle s'exécute avec les paramètres $i=8$ et $tps=20$ lorsqu'elle est appelée sur les paramètres `t`, `e`, `n` et `m` de l'instance donnée en exemple. Remarquons que les deux paramaètres `t` et `e` ne sont jamais modifiés lors des appels récursifs. Nous appelons `ResolPosterCpt` cette nouvelle fonction modifiée.
</div>

In [61]:
cpt = 0
def ResolPosterCpt(t,e,i,tps):
    d = i-1
    global cpt
    if i==8 and tps==20:
        cpt +=1
        
    if i == 0 or tps == 0:
        return 0
    else:
        if t[d] > tps :
            return (ResolPosterCpt(t,e,i-1,tps))
            
        else:
            return max (ResolPosterCpt(t,e,i-1,tps-t[d])+e[d],ResolPosterCpt(t,e,i-1,tps))


ResolPosterCpt(t,e,n,m)
print(cpt)

54


## Résolution du problème: une approche itérative

Afin de ne pas recalculer plusieurs fois la solution à un même sous-problèmes, on propose d'utiliser et de compléter un tableau `Opt` à deux dimension, où `Opt[i,tps]` (pour $ 0 \leq i \leq n$ et $0 \leq tps \leq m$). Ce tableau stocke la valeur maximum de l'enrichissement intellectuel qu'il est possible d’obtenir si l'on se restreint aux $i$ premiers posters ($p_1, p_2, \dots, p_i$) et qu’on s’autorise un temps maximimum de lecture $tps$.

Voici quelques questions à se poser :

1. Quelle valeur devrait être stockée dans `Opt[i, tps]` lorsque $i = 0$ ?
2. Quelle valeur devrait être stockée dans `Opt[i, tps]` lorsque $tps = 0$ ?
3. Supposons maintenant que $i > 0$ et $tps > 0$. Établir une formule (de récurrence) qui donne la valeur de `Opt[i,tps]` en fonction de $t[i]$, de $e[i]$ et de `Opt[i′,tps′]` pour des valeurs plus petites de $i'$ et/ou de $tps'$ ?


<div class="alert alert-danger" role="alert">
Écrire un algorithme qui prend en entrée le nombre $n$ de posters, les listes $t$ et $e$, le temps d’avance $m$ de l’étudiant, puis qui retourne la valeur maximum de l’enrichissement intellectuel qu’il est possible d’obtenir. Pour mettre au point votre fonction, vous pouvez vérifier que le résultat est le même que celui de la fonction récursive préalablement programmée.
</div>

In [116]:
def poster(n,t,e,m):
    Opt=[[0 for _ in range (m+1)]for _ in range (n+1)]
    
    for i in range (1,n+1):
        for tps in range (1,m+1):
            d=i-1
            if t[d] > tps:
                Opt[i][tps] = Opt[d][tps]
            else:
                Opt[i][tps] = max(Opt[d][tps],Opt[d][tps-t[d]]+e[d])
    return Opt[i][tps]            
print (poster(n,t,e,m))

76


<div class="alert alert-danger" role="alert">
Évaluer le temps d’exécution de l'algorithme correspondant à votre fonction `poster`.
</div>

Temps d'éxécution : O(n*m)

<div class="alert alert-danger" role="alert">
Ecrivez une fonction `posterContructif`, qui reprend le code de votre fonction `poster` mais qui retourne également les numéros des posters à lire (et non seulement la valeur de l'enrichissement intellectuel).
</div>

In [127]:
def posterContructif(n,t,e,m):
    Opt=[[0 for _ in range (m+1)]for _ in range (n+1)]
    aLire=[[[] for _ in range (m+1)]for _ in range (n+1)]
    for i in range (1,n+1):
        for tps in range (1,m+1):
            d=i-1
            if t[d] > tps:
                Opt[i][tps] = Opt[d][tps]
                aLire[i][tps] = list (aLire[d][tps])
                
            else:
                Opt[i][tps] = max(Opt[d][tps],Opt[d][tps-t[d]]+e[d])
                if max(Opt[d][tps],Opt[d][tps-t[d]]+e[d])==Opt[d][tps]:
                    aLire[i][tps] = list (aLire[d][tps])
                else:
                    aLire[i][tps] = list (aLire[d][tps-t[d]])
                    aLire[i][tps].append(d)
                    
    return (aLire[i][tps],Opt[i][tps])  
print(posterContructif(n,t,e,m))

([4, 5, 9, 16, 23], 76)


## Plus loin en Python avec les décorateurs

Un **décorateur** est une fonction qui prend en paramètre une autre fonction et qui étend son comportement, sans la modifier. Cela est possible en Python car les fonctions sont des *objets de première classe*: elles peuvent être passées en paramètre (comme le sont les listes, les entiers, les chaînes de caractères ...) à d'autres fonctions. Il est même possible en Python qu'une fonction retourne une fonction. En voici un exemple, où l'on a défini trois fonctions à l'intérieur d'une fonction. Notez que nous avons écrit `return premier` sans parenthèse (ce n'est donc pas un appel de fonction mais bien la fonction qui est retournée).

In [ ]:
def neveuxPicsou(num):
    def premier():
        return "Salut, je suis Riri."

    def second():
        return "Bonjour, mon nom est Fifi !"
    
    def troisieme():
        return "Je me prénomme Loulou."

    if num == 1:
        return premier
    elif num == 2:
        return second
    else:
        return troisieme

In [ ]:
# on obtient une fonction :
neveuxPicsou(2)

In [ ]:
# un appel de fonction
neveuxPicsou(2)()

Lisez maintenant le code suivant :

In [ ]:
def monDecorateur(func):
    def wrapper():
        print("Avant l'appel de fonction.")
        func()
        print("Après l'appel de fonction.")
    return wrapper

def sayHello():
    print("Hello !")

# puis décorons la fonction sayHello pour redéfinir sayHello
sayHello = monDecorateur(sayHello)

In [ ]:
# sayHello désigne maintenant la fonction interne wrapper()
sayHello()

**Sucre syntaxique.** Python propose une façon encore plus commode de définir un décorateur. Regardez l'utilisation de `@...` avant la définition de fonction `sayHello` :

In [ ]:
def monDecorateur(func):
    def wrapper():
        print("Avant l'appel de fonction.")
        func()
        print("Après l'appel de fonction.")
    return wrapper

# la ligne ci-dessous décore la fonction sayHello
@monDecorateur
def sayHello():
    print("Hello !")

sayHello()

** Décorer une fonction ayant des paramètres.** Supposons maintenant que `sayHello` prenne un argument. La solution suivante ne marche pas :

In [4]:
def monDecorateur(func):
    def wrapper():
        print("Avant l'appel de fonction.")
        func()
        print("Après l'appel de fonction.")
    return wrapper

# la ligne ci-dessous décore la fonction sayHello
@monDecorateur
def sayHello(prenom, nom, ):
    print("Hello %s %s!"%(prenom, nom))

sayHello("Bob","Morane")

TypeError: wrapper() takes 0 positional arguments but 2 were given

Observez maintenant l'écriture de la fonction `wrapper` qui récupère bien tous les paramètres pour les donner ensuite à la fonction `func` (la fonction `func` est le paramètre de `monDecorateur`):

In [6]:
def monDecorateur(func):
    def wrapper(param1, param2):
        print("Avant l'appel de fonction.")
        func(param1, param2)
        print("Après l'appel de fonction.")
    return wrapper

# la ligne ci-dessous décore la fonction sayHello
@monDecorateur
def sayHello(prenom, nom, ):
    print("Hello %s %s!"%(prenom, nom))

sayHello("Bob","Morane")

Avant l'appel de fonction.
Hello Bob Morane!
Après l'appel de fonction.


Nous savons maintenant décorer une fonction qui attend des paramètres.

**Comment retourner le résultat d'une fonction décorée ?** Observez plutôt :

In [7]:
def monDecorateur(func):
    def wrapper(param1, param2):
        print("Avant l'appel de fonction.")
        res = func(param1, param2)
        print("Après l'appel de fonction, on retourne le résultat.")
        return res
    return wrapper

# la ligne ci-dessous décore la fonction sayHello
@monDecorateur
def sayHello(prenom, nom, ):
    print("Hello %s %s!"%(prenom, nom))
    return "fin !"

sayHello("Bob","Morane")

Avant l'appel de fonction.
Hello Bob Morane!
Après l'appel de fonction, on retourne le résultat.


'fin !'

Comme vous l'avez observé, nous avons modifiez le `wrapper` pour qu'il retourne le résultat. Le décorateur pourrait lui aussi se saisir du résultat retourné par la fonction décorée pour y effectuer un traitement supplémentaire. C'est ce que nous allons faire dans la suite.

# Utilisation d'un cache pour mémoriser les résultats

Voici une implémentation récursive simple pour calculer le $n$-ième nombre de la [suite de Fibonacci](https://fr.wikipedia.org/wiki/Suite_de_Fibonacci) :

In [ ]:
def fib(n):
    if n==1 or n==2:
        return 1
    else:
        return fib(n-1)+fib(n-2)

Calculons le 12 ième nombre :

In [ ]:
fib(12)

Observons que le calcul de `fib(40)` demande déjà un certain temps (après 1 minute, vous devriez obtenir la réponse, soyez patient ...) :

In [ ]:
fib(40)

Après tout, une fonction est un objet comme un autre en Python. On peut donc ajouter un attribut `cpt` à une fonction `foo`, et ainsi utiliser la variable `foo.cpt`.

In [ ]:
def foo(ch):
    foo.cpt += 1
    print(ch)

foo.cpt = 0
print(foo.cpt)
for fruit in ["banane", "melon", "pomme", "mirabelle"]:
    foo(fruit)
print(foo.cpt)

<div class="alert alert-danger" role="alert">
Écrivez un décorateur `cptRecCall5` qui permettrait de décorer la fonction `fib` et de compter le nombre de fois que `fib` est appelé sur le paramètre $5$ lors d'un appel avec le paramètre $n$.
</div>

In [ ]:
def cptRecCall5(func):
    pass


@cptRecCall5
def fib(n):
    if n==1 or n==2:
        return 1
    else:
        return fib(n-1)+fib(n-2)

<div class="alert alert-danger" role="alert">
Utilisez votre décorateur pour déterminer combien de fois `fib(5)` est exécuté lors d'un appel à `fib(40)` ?
</div>

In [ ]:
pass

réponse : ...

Impressionnant ! On re-calcule autant de fois la même chose. Ce serait beaucoup plus astucieux de ne calculer qu'une seule fois le résultat de `fib(5)`.

Cette fois-ci, plutôt que d'écrire une version itérative de la fonction `fib`, on souhaite ne pas la modifier, garder la version récursive, et écrire un décorateur qui agirait de la façon suivante :
- le décorateur prend une fonction en entrée (par exemple `fib`);
- le décorateur maintient un *cache* à jour;
- lorsque la fonction est appelé avec un certain paramètre, le décorateur vérifie si la fonction a déjà été appelée sur ce paramètre;
- si c'est le cas, le résultat est extrait du cache et retourné;
- si ce n'est pas le cas, la fonction est exécutée sur le paramètre, puis son résultat est stocké dans le cache du décorateur, puis le résultat est retourné.

<div class="alert alert-danger" role="alert">
Inspirez-vous des décorateurs que nous avons donnés pour créer un décorateur `memoize` qui gérerait un tel cache. Un dictionnaire pourrait être une structure de données adaptée pour stocker les informations du cache.
</div>

```
def monDecorateur(func):
    def wrapper(*args, **kwargs):
        print("Avant l'appel de fonction.")
        res = func(*args, **kwargs)
        print("Après l'appel de fonction, on retourne le résultat.")
        return res
    return wrapper
```


In [ ]:
def memoize(func):
    def wrapper_memoize(*args, **kwargs):
        pass
        
    return wrapper_memoize

Une fois le décorateur `memoize` écrit, nous pouvons redéfinir notre fonction fibonnaci. Apprécions aussi la vitesse pour calculer `fib(40)`, alors que la définition de la fonction est restée récursive.

In [ ]:
@memoize
def fib(n):
    if n==1 or n==2:
        return 1
    else:
        return fib(n-1)+fib(n-2)

fib(40)

*remarque:* Si vous avez utilisez une liste ou un dictionnaire comme cache dans `mémoize`, il est possible de l'afficher (`print(fib.cache)`) ou de le vider (`fib.cache.clear()`).


## Retour au problème des posters

Retournons à la fonction récursive `ResolPoster(t,e,n,m)`. Nous pouvons à présent utiliser le mécanisme de cache en décorant cette fonction avec le décorateur `memoize`.

In [ ]:
ResolPoster = memoize(ResolPoster)

Malheureusement, cela ne fonctionne pas tout à fait comme prévu :

In [ ]:
ResolPoster(t,e,n,m)

Manifestement, le problème est que le type `list` ne peut pas être haché; autrement dit, une liste ne peut pas être la clé d'un dictionnaire. Or les paramètres `t` et `e` sont ici des listes.

In [ ]:
monDic = dict()
monDic[3] = "chiffre" # -> OK
monDic["abc"] = "chaîne" # -> OK
monDic[ [1,2,3] ] = "liste" # -> pas possible

Observons que lors de chaque appel récursif, la liste `t` et la liste `e` données en paramètres à la fonction `ResolPoster` ne sont jamais modifiées. Dès lors, ne pourrait-on pas utiliser seulement les deux derniers paramètres (`i ` et `tps`) comme clés pour le cache du décorateur `memoize` ?

<div class="alert alert-danger" role="alert">
Écrivez une nouvelle version du décorateur `memoize` qui utiliserait en entrée du cache uniquement les deux derniers paramètres (correspondant donc à `i` et à `tps`, qui sont des entiers, donc hachables).
</div>

In [ ]:
pass

<div class="alert alert-danger" role="alert">
Écrivez une nouvelle version du décorateur `memoize`. Celui-ci attendrait en paramètre une liste qui contiendrait les paramètres (hachables) de la fonction décorée. Uniquement les paramètres de cette liste serviraient de clés dans le cache (les autres paramètres seraient ignorés).
</div>